In [1]:
%pip install -q -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os
print(load_dotenv('../../.env'))
print(os.environ['LANGSMITH_PROJECT'])

True
agentic-ops


## Setup
- server file: agent.py
- venv is active
- sudo kill -9 $(lsof -t -i:9000)
- langgraph dev --port 9000 --host $(curl -s ipinfo.io/ip)
- langsmith UI introduction

